<a href="https://colab.research.google.com/github/faezesarlakifar/SBU-NLPLab-Internship/blob/main/DataCrawling_ACL_Anthology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [4]:
from bs4 import BeautifulSoup
import json
import numpy as np
import requests
import os 
from tqdm import tqdm
import csv

## Define Constants 

In [3]:
page_url = "https://aclanthology.org/events/acl-2022"
conf_name = 'acl_2022_articles'
conf_id = '2022-acl-long'

In [6]:
html_doc = requests.get(page_url).text
soup = BeautifulSoup(html_doc, 'html.parser')

main_papers = soup.find('div', id = conf_id).find_all('p', class_ = "d-sm-flex")

paper_list = []
counter = 0
for paper_p in main_papers:
    pdf_url = paper_p.contents[0].contents[0]['href']
    paper_span = paper_p.contents[-1]
    assert paper_span.name == 'span'
    paper_a = paper_span.strong.a
    title = paper_a.get_text()
    strs = paper_span.get_text().split('|')
    authors = strs[1:]
    
    
    cleaned_authors = []
    for author in authors:
        author = author.strip('\n')
        cleaned_authors.append(author)
        
    authors = ', '.join(cleaned_authors) 
    
    query = "allintitle: "
    query += title
    
    if(len(cleaned_authors) != 0):
        query += " author: "
        author_ = cleaned_authors[0].split(' ')
        query += author_[0]

    url = "https://aclanthology.org" + paper_a['href']
    
    #TODO find citation

    paper_list.append([title, url, pdf_url, authors])
    

In [7]:
print('There are total {} papers'.format(len(paper_list))) 

There are total 604 papers


## Make a .csv file 

In [8]:
fields = ['Title', 'URL', 'PDF_URL', 'Authors'] 

with open('All_Articles_temp12.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(paper_list)

## Download and save full PDFs

In [10]:
if not os.path.exists(conf_name):
    os.mkdir(conf_name)

illegal_chr = r'\/:*?<>|'
table = ''.maketrans('', '', illegal_chr)
for i, paper in tqdm(list(enumerate(paper_list))):
    r = requests.get(paper[2])
    n = '{}.{}.pdf'.format(i+1, paper[0].translate(table))
    with open('./{}/{}'.format(conf_name, n), 'wb') as f:
        f.write(r.content) 

  0%|          | 1/604 [00:01<17:31,  1.74s/it]


KeyboardInterrupt: ignored